In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib ipympl
# %matplotlib notebook

In [ ]:
args={"SAMPLING_FREQ":10,
      "ROBOT_RADIUS":1
      }

In [ ]:
class TwoWheelRobot:
    """2륜로봇 클래스 
    __init__ : 
        pos ([float,float]): position, 
        ang (float): degree,
        vel ([float,float]): velocity, 
        R (float): radius of robot =0.5 fix 
    """
    def __init__(self, pos=[0,0],vel=[0,0],head=0):
        self.pos = pos
        self.vel = vel
        self.head = head
        self.R = args["ROBOT_RADIUS"]    # robot radius
        self.dt = 1/args["SAMPLING_FREQ"]   # time resolution
        self.traj = []
    
    def set_pos(self,p):
        self.pos = p
        
    def set_vel(self,v):
        self.vel = v 
    
    def get_params(self):
        x,y = self.pos[0], self.pos[1]
        vl,vr = self.vel[0], self.vel[1]  
        head = self.head
        dt = self.dt
        L = 2*self.R    # wheel-base
        
        return x,y,vl,vr,head,dt,L
    
    def set_params(self,x,y,vl,vr,head):
        self.pos = [x,y]
        self.vel = [vl,vr]
        self.head = head 
        
    def move(self):
        x,y,vl,vr,head,dt,L = self.get_params()
        head = np.deg2rad(head)
        
        if vl == vr: 
            x = x + vl*dt*np.cos(head)
            y = y + vr*dt*np.sin(head)
            
        elif vl != vr:
            w = (vr-vl)/L
            rot_R = (L/2)*(vr+vr)/(vr-vl)
            cx = x - rot_R*np.sin(head)
            cy = y + rot_R*np.cos(head)
            
            dh = w*dt 
            dh_rad = dh

            rotate_center = np.array([x-cx,y-cy]).T
            rotate_matrix = np.array([[np.cos(dh_rad), -np.sin(dh_rad)],
                                      [np.sin(dh_rad), np.cos(dh_rad)]])
            
            rotate_w = rotate_matrix @ rotate_center
            
            x = cx + rotate_w[0]
            y = cy + rotate_w[1]
            head += dh
        else: 
            # TODOS: 뭐... 다른 기능을 넣던지.. 
            pass
        
        head = np.rad2deg(head)
        if head>180: 
            head -= 360
        if head<-180:
            head += 360
            
        # update param 
        self.set_params(x,y,vl,vr,head) 
        self.record()
        
    def record(self):
        x,y,_,_,head,_,_ = self.get_params()
        self.traj.append((x,y,head))
    
    def reset(self):
        self.traj = []
    
    def get_record(self):
        return self.traj 
    
    def draw(self,ax,index_=None):
        if index_ is None: 
            x,y,vl,vr,head,dt,L = self.get_params()
        else: 
            x,y,head = self.traj[index_]
            L = self.L

        # draw robot body 
        draw_circle = plt.Circle([x,y], L/2)
        ax.add_patch(draw_circle)
        
        # draw line
        hx = x + L/2*np.cos(np.deg2rad(head))
        hy = y + L/2*np.sin(np.deg2rad(head))
        
        draw_line = plt.Line2D((x,hx),(y,hy),color='r')
        ax.add_line(draw_line)
        

In [ ]:
class Plotter:
    def __init__(self, robot, env=None, obstacles=None):
        self.robot = robot 
        self.env = env 
        self.obstacles = obstacles
        self.f = plt.figure(figsize=(10,10))
        self.ax = self.f.add_subplot(111)
        
    def set_env(self):
        if self.env:
            x_min, x_max, y_min, y_max = self.env.get_area
            self.f.set_xlim = (x_min, x_max)
            self.f.set_ylim = (y_min, y_max)
            # 만들어지면 테스트 
        else: 
            print("env setting please")
            pass
    
    def draw(self, index_=None):
        """각 환경 및 로봇의 위치를 그리게끔 지시
        
        """
        # set aspect: axis equal 
        # self.f = plt.axes(xlim=(-10,10), ylim=(-10,10))
        # self.ax.clear()
        self.ax.set_aspect('equal')
        self.ax.grid()
        self.ax.set_xlim([-10, 10])
        self.ax.set_ylim([-10, 10])
        
        if self.env: 
            self.env.draw(self.ax)
        if self.obstacles:
            self.obstacles.draw(self.ax)
        self.robot.draw(self.ax, index_=index_)
        
        self.ax.set_title("title")
    
    def fig2image(self):
        fig = self.f
        fig.canvas.draw()
        data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
        image = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        return image
        

In [ ]:
import time 

# from moviepy.editor import ImageSequenceClip

robot = TwoWheelRobot(pos=[0,1],vel=[5,5],head=10)
# plotter = Plotter(robot)


fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.grid()


images = []
for iter_ in range(100):
    robot.move()
    
    ax.cla()
    # robot.record()
    x,y,vl,vr,head,dt,L = robot.get_params()
    print(x,y,head)
    
    # draw robot body 
    draw_circle = plt.Circle([x,y], L/2)
    ax.add_patch(draw_circle)
    
    # draw line
    hx = x + L/2*np.cos(np.deg2rad(head))
    hy = y + L/2*np.sin(np.deg2rad(head))
    
    draw_line = plt.Line2D((x,hx),(y,hy),color='r')
    # ax.add_line(draw_line)
    ax.plot([x,hx],[y,hy])
    
    ax.set_xlim([-10, 10])
    ax.set_ylim([-10, 10])
    # plotter.draw()
    # plt.pause(0.01)
    # # images.append(plotter.fig2image())
    
    # robot.draw(ax)
    # ax.imshow(data[i])
    ax.set_title(f"frame {iter_}")
    # Note that using time.sleep does *not* work here!
    plt.pause(0.1)

# ImageSequenceClip(images, fps=5).ipython_display()

In [ ]:
# images[9]

In [ ]:
# animation.save('animation.mp4')

In [ ]:
# HTML(animation.to_html5_video())

In [ ]:
# help(Camera)